# JSON Based Project ( World Bank Data)

Goal:: Using the world bank projects dataset (in file 'dataset/world_bank_projects.json'), provide a solution for below queries.

1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In point 2 above, there are some entries that have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Let's Start 

In [12]:
# loading the neccessary modules and packages

import pandas as pd
import json
from pandas.io.json import json_normalize
from pandas.io.json import to_json

In [13]:
# Reading data into a DataFrame
df= pd.read_json('world_bank_projects.json')

In [14]:
df.shape

(500, 50)

In [15]:
df.head()

,sector,supplementprojectflg,projectfinancialtype,prodline,mjtheme,idacommamt,impagency,project_name,mjthemecode,closingdate,...,majorsector_percent,board_approval_month,theme_namecode,countryname,url,source,projectstatusdisplay,ibrdcommamt,sector_namecode,_id
0,"[{'Name': 'Primary education'}, {'Name': 'Seco...",N,IDA,PE,[Human development],130000000,MINISTRY OF EDUCATION,Ethiopia General Education Quality Improvement...,"8,11",2018-07-07T00:00:00Z,...,"[{'Percent': 46, 'Name': 'Education'}, {'Perce...",November,"[{'code': '65', 'name': 'Education for all'}]",Federal Democratic Republic of Ethiopia,http://www.worldbank.org/projects/P129828/ethi...,IBRD,Active,0,"[{'code': 'EP', 'name': 'Primary education'}, ...",{'$oid': '52b213b38594d8a2be17c780'}
1,[{'Name': 'Public administration- Other social...,N,OTHER,RE,"[Economic management, Social protection and ri...",0,MINISTRY OF FINANCE,TN: DTF Social Protection Reforms Support,"1,6",NaN,...,"[{'Percent': 70, 'Name': 'Public Administratio...",November,"[{'code': '24', 'name': 'Other economic manage...",Republic of Tunisia,http://www.worldbank.org/projects/P144674?lang=en,IBRD,Active,0,"[{'code': 'BS', 'name': 'Public administration...",{'$oid': '52b213b38594d8a2be17c781'}
2,[{'Name': 'Rural and Inter-Urban Roads and Hig...,Y,IDA,PE,"[Trade and integration, Public sector governan...",6060000,MINISTRY OF TRANSPORT AND COMMUNICATIONS,Tuvalu Aviation Investment Project - Additiona...,"5,2,11,6",NaN,...,"[{'Percent': 100, 'Name': 'Transportation'}]",November,"[{'code': '47', 'name': 'Regional integration'...",Tuvalu,http://www.worldbank.org/projects/P145310?lang=en,IBRD,Active,0,"[{'code': 'TI', 'name': 'Rural and Inter-Urban...",{'$oid': '52b213b38594d8a2be17c782'}
3,[{'Name': 'Other social services'}],N,OTHER,RE,"[Social dev/gender/inclusion, Social dev/gende...",0,LABOR INTENSIVE PUBLIC WORKS PROJECT PMU,Gov't and Civil Society Organization Partnership,"7,7",NaN,...,"[{'Percent': 100, 'Name': 'Health and other so...",October,"[{'code': '57', 'name': 'Participation and civ...",Republic of Yemen,http://www.worldbank.org/projects/P144665?lang=en,IBRD,Active,0,"[{'code': 'JB', 'name': 'Other social services'}]",{'$oid': '52b213b38594d8a2be17c783'}
4,[{'Name': 'General industry and trade sector'}...,N,IDA,PE,"[Trade and integration, Financial and private ...",13100000,MINISTRY OF TRADE AND INDUSTRY,Second Private Sector Competitiveness and Econ...,"5,4",2019-04-30T00:00:00Z,...,"[{'Percent': 50, 'Name': 'Industry and trade'}...",October,"[{'code': '45', 'name': 'Export development an...",Kingdom of Lesotho,http://www.worldbank.org/projects/P144933/seco...,IBRD,Active,0,"[{'code': 'YZ', 'name': 'General industry and ...",{'$oid': '52b213b38594d8a2be17c784'}


### Q1. Finding 10 countries with the most number of projects

We wil now look into the various features of the dataset given in the provided "world_bank_projects.json" file and we will also look into any missing values if present and work on it.

In [16]:
# Checking the details of the DataFrames
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
sector                      500 non-null object
supplementprojectflg        498 non-null object
projectfinancialtype        500 non-null object
prodline                    500 non-null object
mjtheme                     491 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
project_name                500 non-null object
mjthemecode                 500 non-null object
closingdate                 370 non-null object
totalcommamt                500 non-null int64
id                          500 non-null object
mjsector_namecode           500 non-null object
docty                       446 non-null object
sector1                     500 non-null object
lendinginstr                495 non-null object
countrycode                 500 non-null object
sector2                     380 non-null object
totalamt                    500 n

In [17]:
# Total number of Countries and Projects

df[['countryname','project_name']].count()

countryname     500
project_name    500
dtype: int64

In [18]:
# Total number of Unique Countries and Projects

df[['countryname', 'project_name']].nunique()

countryname     118
project_name    500
dtype: int64

In [20]:
# Using groupby method to group the data by 'countryname' and sortng it ij descending order in order to get the top countries

countries= df.groupby('countryname')['project_name'].count().sort_values(ascending= False)
countries.head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: project_name, dtype: int64

### Q2.  Find the top 10 major project themes (using column 'mjtheme_namecode') 

In [26]:
# Using a function to flatten the theme data

def theme_flatter():
    themeList = df['mjtheme_namecode'].tolist()
    df_2= [json_normalize(theme) for theme in themeList]
    return pd.concat(df_2, ignore_index = True)

In [27]:
# Flatting the theme data and creating a DataFrame

theme_df = theme_flatter()
theme_df.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [29]:
# Checking for NULL values

theme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
code    1499 non-null object
name    1499 non-null object
dtypes: object(2)
memory usage: 23.5+ KB


In [31]:
# Checking for blank code and name

print('Blank Code: ')
print(theme_df[theme_df.code == ''].count())
print()
print('Blank Name : ')
print(theme_df[theme_df.name== ''].count())

Blank Code: 
code    0
name    0
dtype: int64

Blank Name : 
code    122
name    122
dtype: int64


As, we saw there are 122 entries without names on it for the code. So,we will be ignoring the blank names for the further use.

In [33]:
#Getting top 10 theme names
top_themes = theme_df[theme_df.name!=''].name.value_counts().sort_values(ascending= False)
top_themes.head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Name: name, dtype: int64

### Q3.  In query 2 above, there are some theme observations where the name is missing. Create a dataframe with the missing names filled in.

In [35]:
# Finding out the relation between the names and coses in order to fill the missing project names based on the code.
# Total number of unique codes available in the Dataframe

theme_df['code'].nunique()

11

As we observed we have 11 unique codes for the project themes.So, let's find out the corresponding project names.

In [42]:
# Finding the total unique combination of code and name
unique_theme_df = theme_df.drop_duplicates()
unique_theme_df = unique_theme_df[unique_theme_df.name != '']

unique_theme_df = unique_theme_df.reset_index().drop('index', axis=1)
unique_theme_df


,code,name
0,8,Human development
1,1,Economic management
2,6,Social protection and risk management
3,5,Trade and integration
4,2,Public sector governance
5,11,Environment and natural resources management
6,7,Social dev/gender/inclusion
7,4,Financial and private sector development
8,10,Rural development
9,9,Urban development


Now we have a new DataFrame with unqiue code and name combination.We will be merging both the raw and new theme DataFrame to fill all the blank project names.

In [45]:
new_theme_df = pd.merge(theme_df, unique_theme_df, on='code', suffixes=('_x','')).drop('name_x', axis=1)
new_theme_df.head()

,code,name
0,8,Human development
1,8,Human development
2,8,Human development
3,8,Human development
4,8,Human development


In [46]:
new_theme_df['name'].value_counts()

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: name, dtype: int64

In [48]:
# Now we need to verify that there are no more blank values for theme name
new_theme_df[new_theme_df.name ==''].count()

code    0
name    0
dtype: int64

In [ ]:
So, here we observe no more blank values are left with names. Therefore the solutions stands complete. 